<a href="https://colab.research.google.com/github/kiran-pyt/Back-pro/blob/main/S4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.block0 = nn.Sequential(
          nn.Conv2d(1, 16, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout(0.1),
        )
        self.block1 = nn.Sequential(
          nn.Conv2d(16, 16, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(16),
        )
        self.block1d = nn.Sequential(
          nn.Conv2d(16, 16, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout(0.1),
        )
        self.block2 = nn.Sequential(
          nn.Conv2d(16, 32, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(32),
          nn.Dropout(0.1),
        )
        self.block3 = nn.Sequential(
          nn.Conv2d(32, 16, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(16),
        )
        self.block4 = nn.Sequential(
          nn.Conv2d(16, 10, 1, bias=False),
        )
        self.pool1 = nn.AvgPool2d(2, 2)
        self.pool2 = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        x = self.pool1(self.block1(self.block1d(self.block0(x))))
        x = self.block4(self.pool2(self.block3(self.block2(x))))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:

!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 16, 22, 22]           2,304
             ReLU-10           [-1, 16, 22, 22]               0
      BatchNorm2d-11           [-1, 16, 22, 22]              32
        AvgPool2d-12           [-1, 16, 11, 11]               0
    

<ipython-input-2-beb637460878>:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:


torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

     

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
     

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]<ipython-input-2-beb637460878>:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.07236165553331375 batch_id=1874: 100%|██████████| 1875/1875 [01:05<00:00, 28.47it/s]



Test set: Average loss: 0.0490, Accuracy: 9858/10000 (98.58%)



loss=0.018083767965435982 batch_id=1874: 100%|██████████| 1875/1875 [01:06<00:00, 28.07it/s]



Test set: Average loss: 0.0345, Accuracy: 9898/10000 (98.98%)



loss=0.05571356043219566 batch_id=1874: 100%|██████████| 1875/1875 [01:07<00:00, 27.83it/s]


...


Test set: Average loss: 0.0167, Accuracy: 9955/10000 (99.55%)



loss=0.0003303311241324991 batch_id=1874: 100%|██████████| 1875/1875 [01:07<00:00, 27.71it/s]



Test set: Average loss: 0.0174, Accuracy: 9939/10000 (99.39%)



loss=0.0007995847263373435 batch_id=1874: 100%|██████████| 1875/1875 [01:08<00:00, 27.50it/s]



Test set: Average loss: 0.0191, Accuracy: 9935/10000 (99.35%)

